In [238]:
from datetime import datetime
import sqlite3
from uuid import uuid4

In [259]:
class Database:
    """this is a database class - current 1hr"""
    def __init__(self, dbfile):
        """initialize the db class"""
        self.dbfile = dbfile
        
    def run_command(self, command):
        """runs a command and commits changes"""
        db = sqlite3.connect(self.dbfile)
        c = db.cursor()
        c.execute(command)
        db.commit()
        db.close()
        
    def dict_to_sql(self, dictionary):
        """changes a dictionary into a sql data injection"""
        return ", ".join([f"{key}='{value}'" for key, value in dictionary.items()]) if dictionary else ''
        
    def make_query(self, query):
        """changes a dictionary query into an AND statement query"""
        string_comm = " AND ".join([f"{key}='{value}'" for key, value in query.items()]) if query else ''
        return string_comm
        
    def or_query(self, queries):
        """changes an array of dictionaries into OR and AND statement queries"""
        string_comm = " OR ".join([self.make_query(query) for query in queries])
        return string_comm
        
    def create_table(self, name, structure):
        """creates a new table"""
        structure = ', '.join([f'{key} {value}' for key, value in structure.items()])
        command = f'CREATE TABLE IF NOT EXISTS {name} ({structure})'
        self.run_command(command)
        
    def post(self, tablename, payload):
        """creates a new row in the table"""
        data = ", ".join([f"'{value}'" for key, value in payload.items()])
        cols = ", ".join([f"{key}" for key, value in payload.items()])
        command = f'INSERT INTO {tablename} ({cols}) VALUES({data})'
        
        self.run_command(command)
        
    def get(self, tablename, queries=None, cols='*'):
        """gets a list of arrays from a specific query"""
        db = sqlite3.connect(self.dbfile)
        c = db.cursor()
        query_string = 'WHERE ' + self.or_query(queries) if len(queries) > 0 else ''
        command = f'SELECT {cols} FROM {tablename} ' + query_string
        c.execute(command)
        query = c.fetchall()
        db.close()
        return query
    
    def put(self, tablename, payload, queries):
        """changes queries to the payload"""
        if len(queries) > 0:
            query_string = 'WHERE ' + self.or_query(queries)
            command = f"UPDATE {tablename} SET {self.dict_to_sql(payload)}" + query_string
            self.run_command(command)
        
    def delete(self, tablename, queries):
        """deletes a query"""
        
        print([*map(lambda x: x, queries)])
        
        if len(queries) > 0:
            query_string = "WHERE " + self.or_query(queries) if len(queries) > 0 else ''
            command = f"DELETE FROM {tablename} " + query_string
            self.run_command(command)
        
db = Database('example.db')
db.create_table('notes',{
    "id":"Text",
    "timestamp":"Date",
    "content":"Text",
    "is_action":"Boolean",
    "tags":"Text",
    "completed":"Boolean"
})
db.delete('notes',[{}])

[{}]


OperationalError: incomplete input

In [249]:
def delete_note(db, ids=[]):
    if len(ids) > 0:
        comm = [{'id':x} for x in ids]
        db.delete('notes', comm)
        return f'deleted {ids}'
    else:
        return f'nothing was deleted'

print(delete_note(db))
delete_note(db,[
    'df1bbbd5-b118-4238-97b4-2b0fd6968f1a',
    '8232e699-3eaf-4de6-a6cf-06e054b5f91c'
])

In [253]:
def new_note(db, content, is_action, tags, date=datetime.now()):
    """creates a new note entry that defaults to the current date"""
    new_post = {
        "id":uuid4(),
        "timestamp":date,
        "content":content,
        "is_action":is_action,
        "tags":';'.join(tags)
    }
    
    db.post('notes', new_post)
    new_data = db.get('notes', queries=[new_post])
    return new_data

# new_note(db, 'this is a new thing', True, ['cool'])

In [254]:
def get_note(db, queries=None):
    return db.get('notes', queries=queries)

# get_note(db, [{'is_action':True}])

In [255]:
def edit_note(db, changes, ids=[]):
    if len(ids) > 0:
        db.put('notes', changes, [{'id':x for x in ids}])
        return db.get('notes', queries=[{'id': x for x in ids}] )
    else: 
        return 'nothing was changed'
    
# print(edit_note(db, {'content':'hello...'}))
# print(edit_note(db, {'content':'this is not a hello world anymore'}, ids=['93061bd8-7402-40fa-81d8-47e0b058a1ca']))